In [1]:
import pandas as pd
import numpy as np
import os 
import datetime
from rich.progress import track

In [2]:
# 184 220739
GCS_eye = {
    "Spontaneously": 4,
    "To Speech": 3,
    "To Pain": 2,
    "None": 1,
}
# 454 223901
GCS_motor = {
    "Obeys Commands": 6,
    "Localizes Pain": 5,
    "Flex-withdraws": 4,
    "Abnormal Flexion": 3,
    "Abnormal extension": 2,
    "No response":1,    
}
# 723 223900
# 1.0 ET/Trach, 1 No Response, 2 Incomp sounds, 3 Inapprop words, 4 Confused, 5 Oriented
GCS_verbal = {
    "Oriented": 5,
    "Confused": 4,
    "Inappropriate Words": 3,
    "Incomprehensible sounds": 2,
    "No Response-ETT": 1,
    "No Response": 1,
}

In [34]:
# IABP Mean [224, 224322]
# pCO2_Blood_Blood Gas [50818]
# Platelets [828, 30006, 225170]

mapping_dict = {
    "Albumin": [50862],
    "ALP": [50863],
    "ALT": [50861],
    "AST": [50878],
    "Bilirubin": [50885],
    "BUN": [51006],
    "Cholesterol": [50907],
    "Creatinine": [50912],
    "DiasABP": [8368,8555,220051],
    "FiO2": [2981, 3420, 3422, 223835],
    "GCS": [198,223901,223900,220739], # 198 = 184 + 454 + 723
    "Glucose": [807,811,1529,3745,3744,225664,220621,226537,50809,50931],
    "HCO3": [50882],
    "HCT": [51221],
    "HR": [211, 220045],
    "K": [50971],
    "Lactate": [50813],
    "Mg": [50960],
    "MAP": [52,6702,220052],
    "Na": [50983],
    "NIDiasABP": [8441,220180],
    "NIMAP":[456, 220181],
    "NISysABP": [455,220179],
    "PaCO2": [778,220235,50818],
    "PaO2": [779,220224,50821],
    "pH": [780,1126,4753,223830],
    "Platelets": [51265],
    "RespRate": [618, 615, 220210, 224690],
    "SaO2": [834, 220227],
    "SysABP": [51,6701,220050],
    "Temp": [676, 223762],
    "TropI": [51002],
    "TropT": [51003],
    "Urine": [40055, 226559],
    "WBC": [51301],
    "Weight": [763, 224639],
    "Mech": [651],
}
selected_itemid = []
for k,v in mapping_dict.items():
    selected_itemid.extend(v)

time_features = list(mapping_dict.keys())
    
len(selected_itemid),len(mapping_dict),time_features

(76,
 37,
 ['Albumin',
  'ALP',
  'ALT',
  'AST',
  'Bilirubin',
  'BUN',
  'Cholesterol',
  'Creatinine',
  'DiasABP',
  'FiO2',
  'GCS',
  'Glucose',
  'HCO3',
  'HCT',
  'HR',
  'K',
  'Lactate',
  'Mg',
  'MAP',
  'Na',
  'NIDiasABP',
  'NIMAP',
  'NISysABP',
  'PaCO2',
  'PaO2',
  'pH',
  'Platelets',
  'RespRate',
  'SaO2',
  'SysABP',
  'Temp',
  'TropI',
  'TropT',
  'Urine',
  'WBC',
  'Weight',
  'Mech'])

In [21]:
valid_range = {
     'Albumin': {'min': 0, 'max': 6},
     'ALP': {'min': 0, 'max': 5000},
     'ALT': {'min': 0, 'max': 25000},
     'AST': {'min': 0, 'max': 40000},
     'Bilirubin': {'min': 0, 'max': 100},
     'BUN': {'min': 0, 'max': 250},
     'Cholesterol': {'min': 0, 'max': 800},
     'Creatinine': {'min': 0, 'max': 25},
     'DiasABP': {'min': 0, 'max': 300},
     'FiO2': {'min': 0, 'max': 100},
     'GCS': {'min': 3, 'max': 15},
     'Glucose': {'min': 0, 'max': 1600},
     'HCO3': {'min': 0, 'max': 60},
     'HCT': {'min': 0, 'max': 70},
     'HR': {'min': 0, 'max': 350},
     'K': {'min': 0, 'max': 30},
     'Lactate': {'min': 0, 'max': 40},
     'Mg': {'min': 0, 'max': 40},
     'MAP': {'min': 0, 'max': 375},
     'Na': {'min': 0, 'max': 200},
     'NIDiasABP': {'min': 0, 'max': 300}, # !!!
     'NIMAP': {'min': 0, 'max': 300},
     'NISysABP': {'min': 0, 'max': 300},
     'PaCO2': {'min': 0, 'max': 200},
     'PaO2': {'min': 0, 'max': 800},
     'pH': {'min': 6, 'max': 8},
     'Platelets': {'min': 0, 'max': 2500},
     'RespRate': {'min': 0, 'max': 100},
     'SaO2': {'min': 0, 'max': 100},
     'SysABP': {'min': 0, 'max': 300},
     'Temp': {'min': 15, 'max': 47},
     'TropI': {'min': 0, 'max': 50},
     'TropT': {'min': 0, 'max': 30},
     'Urine': {'min': 0, 'max': 3000},
     'WBC': {'min': 0, 'max': 550},
     'Weight': {'min': 20, 'max': 300}, # !!!
     'Mech': {'min': 0, 'max': 100}
    }

In [46]:
re_mapping_dict = {}
count = 0
for k,v in mapping_dict.items():
    for i in v:
        re_mapping_dict[i] = count
    count+=1
re_mapping_dict

{50862: 0,
 50863: 1,
 50861: 2,
 50878: 3,
 50885: 4,
 51006: 5,
 50907: 6,
 50912: 7,
 8368: 8,
 8555: 8,
 220051: 8,
 2981: 9,
 3420: 9,
 3422: 9,
 223835: 9,
 198: 10,
 223901: 10,
 223900: 10,
 220739: 10,
 807: 11,
 811: 11,
 1529: 11,
 3745: 11,
 3744: 11,
 225664: 11,
 220621: 11,
 226537: 11,
 50809: 11,
 50931: 11,
 50882: 12,
 51221: 13,
 211: 14,
 220045: 14,
 50971: 15,
 50813: 16,
 50960: 17,
 52: 18,
 6702: 18,
 220052: 18,
 50983: 19,
 8441: 20,
 220180: 20,
 456: 21,
 220181: 21,
 455: 22,
 220179: 22,
 778: 23,
 220235: 23,
 50818: 23,
 779: 24,
 220224: 24,
 50821: 24,
 780: 25,
 1126: 25,
 4753: 25,
 223830: 25,
 51265: 26,
 618: 27,
 615: 27,
 220210: 27,
 224690: 27,
 834: 28,
 220227: 28,
 51: 29,
 6701: 29,
 220050: 29,
 676: 30,
 223762: 30,
 51002: 31,
 51003: 32,
 40055: 33,
 226559: 33,
 51301: 34,
 763: 35,
 224639: 35,
 651: 36}

In [23]:
os.listdir("./")

['preprocess_MIMIC3.ipynb', 'data', 'mimic3-benchmarks', '.ipynb_checkpoints']

In [7]:
mimic3_path = "xxxxx/MIMIC_III/full_data/"
file_path = "xxxxx/MIMIC_III/mimic3-benchmarks/data/"
data_path = "./data/"
if not os.path.exists(data_path):
    os.mkdir(data_path)
if not os.path.exists(mimic3_path):
    raise FileNotFoundError("MIMIC3 csv data not founded in: %s", mimic3_path)
else:
    print(os.listdir(mimic3_path))

['.DS_Store', 'PRESCRIPTIONS.csv', 'D_ICD_DIAGNOSES.csv', 'D_ICD_PROCEDURES.csv', 'CHARTEVENTS.csv', 'SERVICES.csv', 'PROCEDURES_ICD.csv', 'PATIENTS.csv', 'MICROBIOLOGYEVENTS.csv', 'ADMISSIONS.csv', 'CALLOUT.csv', 'CAREGIVERS.csv', 'robots.txt.tmp', 'TRANSFERS.csv', 'D_CPT.csv', 'NOTEEVENTS.csv', 'DATETIMEEVENTS.csv', 'ICUSTAYS.csv', 'DRGCODES.csv', 'LABEVENTS.csv', 'CPTEVENTS.csv', 'DIAGNOSES_ICD.csv', 'D_ITEMS.csv', 'D_LABITEMS.csv', 'PROCEDUREEVENTS_MV.csv', 'OUTPUTEVENTS.csv', 'INPUTEVENTS_MV.csv', 'INPUTEVENTS_CV.csv']


In [ ]:
train_index = list(os.listdir(file_path + "/train/"))
test_index = list(os.listdir(file_path + "/test/"))

patient_files = []
for sub_index in train_index:
    patient_files.append(["xxxxx/MIMIC_III/mimic3-benchmarks/data/train/",sub_index])
for sub_index in test_index:
    patient_files.append(["xxxxx/MIMIC_III/mimic3-benchmarks/data/test/",sub_index])
len(patient_files), patient_files[:5]

In [24]:
exp_path = patient_files[1][0]+patient_files[1][1] + "/"
os.listdir(exp_path)

['stays.csv',
 'diagnoses.csv',
 'episode3.csv',
 'episode3_timeseries.csv',
 'episode2_timeseries.csv',
 'events.csv',
 'episode2.csv']

In [57]:
def load_stay(exp_path):
    exp_stays = pd.read_csv(exp_path+"/stays.csv")
    exp_stays['INTIME'] = pd.to_datetime(exp_stays['INTIME'])
    exp_stays['DISCHTIME'] = pd.to_datetime(exp_stays['DISCHTIME'])
    exp_stays['LOS2'] = round((exp_stays['DISCHTIME']-exp_stays['INTIME'])/datetime.timedelta(days=1),4)
    out_stay = []
    for index,row in exp_stays.iterrows():
        if row.LOS < 2 or np.isnan(row.LOS):
            continue            
        out_stay.append([index, row.HADM_ID,row.INTIME, [row.MORTALITY_INUNIT,row.MORTALITY_INHOSPITAL,row.MORTALITY,row.LOS,row.LOS2]])
    return out_stay

exp_stays = load_stay(exp_path)
exp_stays

[[1, 182917, Timestamp('2125-08-03 07:35:27'), [0, 0, 0, 4.1627, 10.274]],
 [2, 188422, Timestamp('2125-08-25 12:06:45'), [0, 0, 0, 13.0287, 20.1828]]]

In [27]:
def is_numerical(df,col_name):
    df['CHECK'] = df.VALUE.apply(lambda x:str(x).replace(".",'',1).isdigit())
    return df

def transfer_gcs(df):
#     print(len(df[(df.ITEMID == 223901) | (df.ITEMID == 223900) | (df.ITEMID == 220739)]))
#     df.CHARTTIME = pd.to_datetime(df.CHARTTIME)
#     print(df[(df.ITEMID == 223901) | (df.ITEMID == 223900) | (df.ITEMID == 220739)].sort_values("CHARTTIME")[:10])
    df.VALUE = df.apply(lambda row: GCS_eye[row.VALUE] if row.ITEMID == 220739 else row.VALUE,axis=1)
    df.VALUE = df.apply(lambda row: GCS_motor[row.VALUE] if row.ITEMID == 223901 else row.VALUE,axis=1)
    df.VALUE = df.apply(lambda row: GCS_verbal[row.VALUE] if row.ITEMID == 223900 else row.VALUE,axis=1)
    return df

def load_event(exp_path):
    exp_events = pd.read_csv(exp_path+"/events.csv")
    exp_events.CHARTTIME = pd.to_datetime(exp_events.CHARTTIME)
    exp_events = exp_events[~exp_events.VALUE.isnull()]
    exp_events = transfer_gcs(exp_events)
    exp_events = is_numerical(exp_events,'VALUE')
    return exp_events[exp_events.CHECK]# .drop(['CHECK'],1)

exp_events = load_event(exp_path)
exp_events

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,CHECK
4,1050,182917,229166,2125-08-03 12:00:00,52,70,mmHg,True
5,1050,182917,229166,2125-08-03 12:00:00,55,84,mmHg,True
6,1050,182917,229166,2125-08-03 12:00:00,59,101,mmHg,True
7,1050,182917,229166,2125-08-03 12:00:00,62,52,mmHg,True
17,1050,182917,229166,2125-08-03 12:00:00,211,77,BPM,True
...,...,...,...,...,...,...,...,...
17982,1050,182917,229166,2125-08-05 23:00:00,40069,200,ml,True
17983,1050,188422,248879,2125-08-25 19:00:00,40055,35,ml,True
17984,1050,188422,248879,2125-08-29 08:00:00,40060,0,NaN,True
17985,1050,188422,248879,2125-08-26 10:00:00,40055,45,ml,True


In [28]:
def divide_event_by_stay(event,adm_id,intime):
    event = event[event.HADM_ID == adm_id]
    event.CHARTTIME = (event.CHARTTIME - intime) / datetime.timedelta(hours=1)
    event = event.sort_values("CHARTTIME")
    event = admission_in_hours(event,"CHARTTIME",48)
    return event

def admission_in_hours(df,col_name,hours=48):
    df = df[(df[col_name]>=0)&(df[col_name]<48)]
    return df

for admission in exp_stays:
    print(admission)
    exp_out_events = divide_event_by_stay(exp_events,admission[1],admission[2])
    break
    
exp_out_events

[1, 182917, Timestamp('2125-08-03 07:35:27')]


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,CHECK
9589,1050,182917,229166,0.909167,742,1,kg,True
14260,1050,182917,229166,0.909167,8547,160,mmHg,True
14262,1050,182917,229166,0.909167,8551,165,mmHg,True
14263,1050,182917,229166,0.909167,8552,20,mmHg,True
14264,1050,182917,229166,0.909167,8553,30,BPM,True
...,...,...,...,...,...,...,...,...
10967,1050,182917,229166,47.409167,5819,8,BPM,True
10968,1050,182917,229166,47.409167,5820,92,%,True
1090,1050,182917,229166,47.409167,211,96,BPM,True
1098,1050,182917,229166,47.409167,456,63,mmHg,True


In [80]:
def get_episode_info(exp_path, e_index):
    # print(os.listdir(exp_path))
    if not os.path.exists(exp_path+"/episode%d.csv"%e_index):
        return []
    raw_info = pd.read_csv(exp_path+"/episode%d.csv"%e_index)
    raw_info = np.array(raw_info)[0]
    info = np.full(12,-1.0)
    info[int(raw_info[1])] = 1
    info[5 + int(raw_info[2])] = 1
    for index in range(3,6):
        if not np.isnan(raw_info[index]):
            info[9+index-3]=raw_info[index]
    diag = np.array(raw_info[8:], dtype=np.int)
    return info, diag

for admission in exp_stays:
    print(admission)
    exp_infos = get_episode_info(exp_path,admission[0]+1)
    break
exp_infos

[1, 182917, Timestamp('2125-08-03 07:35:27'), [0, 0, 0, 4.1627, 10.274]]


(array([-1.        , -1.        , -1.        , -1.        ,  1.        ,
        -1.        , -1.        ,  1.        , -1.        , 84.93785283,
        -1.        , -1.        ]),
 array([1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [60]:
# col_name = ["Hour"] + list(mapping_dict.keys())
def generate_sample(adm_events):
    adm_events = adm_events[adm_events.ITEMID.isin(selected_itemid)]
    
    sample_re_stime = [3600*i for i in range(48)]
    sample_re_tdata = np.full((48,len(time_features)),0.0)
    sample_re_tmask = np.full((48,len(time_features)),0.0)
    
    sample_ir_stime = list(set(adm_events.CHARTTIME))
    sample_ir_stime.sort()
    sample_ir_tdata = []
    sample_ir_tmask = []
    
    for t in sample_ir_stime:
        sub_data = np.full(len(time_features), 0.0)
        sub_mask = np.full(len(time_features), 0)
        sub_events = adm_events[adm_events.CHARTTIME==t]
        t_hour = int(t)
        for index,row in sub_events.iterrows():
            f_index = re_mapping_dict[row.ITEMID]
            feature_name = time_features[f_index]
            min_feature = valid_range[feature_name]["min"]
            max_feature = valid_range[feature_name]["max"]
            
            if float(row.VALUE)>=min_feature and float(row.VALUE)<=max_feature:    
                if f_index == 11: # GCS
                    if sub_data[f_index]==0.0:
                        sub_data[f_index] = float(row.VALUE)
                        sub_mask[f_index] = 1
                        sample_re_tdata[t_hour, f_index]=float(row.VALUE)
                        sample_re_tmask[t_hour, f_index]=1
                    else:
                        sub_data[f_index] += float(row.VALUE)
                        sample_re_tdata[t_hour, f_index]=float(row.VALUE)
                else:   
                    sub_data[f_index] = float(row.VALUE)
                    sub_mask[f_index] = 1
                    sample_re_tdata[t_hour, f_index]=float(row.VALUE)
                    sample_re_tmask[t_hour, f_index]=1
        sample_ir_tdata.append(sub_data)
        sample_ir_tmask.append(sub_mask)
    
    
    return [{
                 "stime": sample_re_stime,
                 "tdata": sample_re_tdata,
                 "tmask": sample_re_tmask,
             },
             {
                 "stime": sample_ir_stime,
                 "tdata": np.array(sample_ir_tdata),
                 "tmask": np.array(sample_ir_tmask),
             }]

tem = generate_sample(exp_out_events)
tem # , np.sum(~np.isnan(tem),1)

[{'stime': [0,
   3600,
   7200,
   10800,
   14400,
   18000,
   21600,
   25200,
   28800,
   32400,
   36000,
   39600,
   43200,
   46800,
   50400,
   54000,
   57600,
   61200,
   64800,
   68400,
   72000,
   75600,
   79200,
   82800,
   86400,
   90000,
   93600,
   97200,
   100800,
   104400,
   108000,
   111600,
   115200,
   118800,
   122400,
   126000,
   129600,
   133200,
   136800,
   140400,
   144000,
   147600,
   151200,
   154800,
   158400,
   162000,
   165600,
   169200],
  'tdata': array([[ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
         [ 0. , 95. , 53. , ..., 16.4,  0. ,  0. ],
         [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
         ...,
         [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
         [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
         [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ]]),
  'tmask': array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 1., 1., ..., 1., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0.

In [87]:
count = 0
for file in track(patient_files):
    # print(os.listdir(file[0]+file[1]))
    exp_stays = load_stay(file[0]+file[1])
    if len(exp_stays) == 0 :
        # print("no valid stay ",file[1])
        continue
    exp_events = load_event(file[0]+file[1])
    if len(exp_events)==0:
        # print("no valid event ",file[1])
        continue

    for admission in exp_stays:
        # print(admission)
        count +=1
        adm_data = divide_event_by_stay(exp_events,admission[1],admission[2])
        if len(adm_data) == 0:
            print("no event after division ",file[1],admission[1], admission[2])
            continue
        exp_infos = get_episode_info(file[0]+file[1], admission[0]+1)
        if exp_infos == []:
            print("no info and diag ",file[1],admission[0]+1, admission[2])
            continue
        adm_data = generate_sample(adm_data)
        
        np.savez(data_path+"%s_episode%d"%(file[1],admission[0]),
                 info = exp_infos[0],
                 diag = exp_infos[1],
                 regular_data = adm_data[0],
                 irregular_data = adm_data[1],
                 labels = admission[3],
                )

Output()

no event after division  14761 155581 2193-05-31 04:57:00
no event after division  14761 107155 2193-07-11 21:17:00
no event after division  5772 133327 2163-07-30 23:05:00
no event after division  16881 166063 2164-08-17 14:10:00
no event after division  23761 111184 2187-09-08 21:27:00
no event after division  24552 124040 2152-12-08 06:17:00
no event after division  4264 147554 2107-07-10 14:07:00
no event after division  2592 130856 2183-06-05 21:03:00
no event after division  5259 195496 2144-03-08 16:38:00
no event after division  23946 197173 2148-07-01 19:05:00
no event after division  29280 134662 2159-07-30 13:19:00
no event after division  2322 169179 2176-03-14 09:25:00
no event after division  87428 116173 2118-01-15 11:43:00
no event after division  9736 106592 2184-06-06 09:59:00
no event after division  7534 194035 2160-05-18 17:44:00
no event after division  30610 180378 2165-05-07 02:27:00
no event after division  25115 153867 2198-05-03 15:28:00
no event after divisi

In [88]:
tem = np.load(data_path+"1050_episode1.npz")
print(list(tem.keys()))
for key in tem.keys():
    print(tem[key])

['info', 'diag', 'regular_data', 'irregular_data', 'labels']
[-1.         -1.         -1.         -1.          1.         -1.
 -1.          1.         -1.         84.93785283 -1.         -1.        ]
[1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
{'stime': [0, 3600, 7200, 10800, 14400, 18000, 21600, 25200, 28800, 32400, 36000, 39600, 43200, 46800, 50400, 54000, 57600, 61200, 64800, 68400, 72000, 75600, 79200, 82800, 86400, 90000, 93600, 97200, 100800, 104400, 108000, 111600, 115200, 118800, 122400, 126000, 129600, 133200, 136800, 140400, 144000, 147600, 151200, 154800, 158400, 162000, 165600, 169200], 'tdata': array([[ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. , 95. , 53. , ..., 16.4,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       ...,
      

In [94]:
files_list = os.listdir(data_path)
files = []
for file in files_list:
    if file[-3:]=="npz":
        files.append(file)
files.sort()
assert len(files)==21139,"[x] Error: Please preprocess files again!"

len(files),files[:5]

(21139,
 ['10004_episode0.npz',
  '10007_episode0.npz',
  '1000_episode0.npz',
  '10011_episode0.npz',
  '10013_episode0.npz'])

In [95]:
re_samples_data = []
re_samples_mask = []

ir_samples_data = []
ir_samples_mask = []

info_list = []
label_list = []

for file in files:
    data = np.load(data_path+"%s.npz"%(file[:-4]))
    info_list.append(data['info'])
    label_list.append(data['labels'])
    re_samples_data.append(data['regular_data'][()]['tdata'])
    re_samples_mask.append(data['regular_data'][()]['tmask'])
    ir_samples_data.append(data['irregular_data'][()]['tdata'])
    ir_samples_mask.append(data['irregular_data'][()]['tmask'])

In [108]:
info_dict = {}
for i in range(12):
    info_dict[i] = []
for sub_info in info_list:
    for i in range(12):
        if sub_info[i] != -1:
            info_dict[i].append(sub_info[i])

info_result = []
for i in range(12):
    info_result.append(len(info_dict[i]))
print(info_result)

count = 0
for i in range(5):
    count+=len(info_dict[i])
print(demo_features[0],count,round(count/211.39,2))
count = 0
for i in range(6,8):
    count+=len(info_dict[i])
print(demo_features[1],count,round(count/211.39,2))
for i in range(9,12):
    count = len(info_dict[i])
    print(demo_features[i-7],count,round(count/211.39,2))

[2909, 492, 2016, 679, 15043, 0, 9510, 11629, 0, 21139, 5370, 18181]
Ethnicity 21139 100.0
Gender 21139 100.0
Age 21139 100.0
Height 5370 25.4
Weight 18181 86.01


In [127]:
# row.MORTALITY_INUNIT,row.MORTALITY_INHOSPITAL,row.MORTALITY,row.LOS,row.LOS2
label_list=np.array(label_list)
y1 = np.array(label_list[:,1],int)
sum(y1)

(2797, 1238, 9175)

In [128]:
from sklearn.model_selection import train_test_split, KFold
def get_folds(indices, array_y, num=5):
    folds = []
    for i in range(num):
        index_train, index_valid_test, Y_train, Y_valid_test = train_test_split(indices, array_y, test_size=0.2, random_state=2012+i,
                                                               shuffle=True,stratify=array_y)
        index_valid, index_test, Y_valid, Y_test = train_test_split(index_valid_test, Y_valid_test, test_size=0.5, random_state=2012+i,
                                                               shuffle=True,stratify=Y_valid_test)

        print("[train] 1:%d 0:%d all:%d" %(sum(Y_train),len(Y_train)-sum(Y_train),len(Y_train)))
        print("[valid] 1:%d 0:%d all:%d" %(sum(Y_valid),len(Y_valid)-sum(Y_valid),len(Y_valid)))
        print("[test]  1:%d 0:%d all:%d" %(sum(Y_test),len(Y_test)-sum(Y_test),len(Y_test)))
        folds.append([index_train,index_valid,index_test])
    return folds

In [129]:
folds1 = get_folds(np.array(files), y1)
folds1

[train] 1:2238 0:14673 all:16911
[valid] 1:280 0:1834 all:2114
[test]  1:279 0:1835 all:2114
[train] 1:2238 0:14673 all:16911
[valid] 1:280 0:1834 all:2114
[test]  1:279 0:1835 all:2114
[train] 1:2238 0:14673 all:16911
[valid] 1:280 0:1834 all:2114
[test]  1:279 0:1835 all:2114
[train] 1:2238 0:14673 all:16911
[valid] 1:280 0:1834 all:2114
[test]  1:279 0:1835 all:2114
[train] 1:2238 0:14673 all:16911
[valid] 1:279 0:1835 all:2114
[test]  1:280 0:1834 all:2114


[[array(['7825_episode0.npz', '22816_episode0.npz', '16748_episode0.npz',
         ..., '45374_episode0.npz', '17707_episode0.npz',
         '7666_episode0.npz'], dtype='<U19'),
  array(['72483_episode0.npz', '17668_episode0.npz', '85272_episode0.npz',
         ..., '18123_episode2.npz', '31361_episode0.npz',
         '25723_episode0.npz'], dtype='<U19'),
  array(['5760_episode0.npz', '17859_episode1.npz', '82960_episode0.npz',
         ..., '23118_episode0.npz', '6128_episode0.npz',
         '24981_episode1.npz'], dtype='<U19')],
 [array(['16687_episode2.npz', '27598_episode0.npz', '99256_episode0.npz',
         ..., '3866_episode6.npz', '20711_episode0.npz',
         '29514_episode0.npz'], dtype='<U19'),
  array(['30020_episode0.npz', '43004_episode0.npz', '24746_episode0.npz',
         ..., '18483_episode0.npz', '85541_episode0.npz',
         '23849_episode0.npz'], dtype='<U19'),
  array(['64082_episode0.npz', '26043_episode0.npz', '25966_episode0.npz',
         ..., '13033_episode4

In [138]:
# demographic infomation norm
def get_norm_demo(info_list,folds):
    print("============== norm of demographic features =================")
#     info_list = []
#     for file in track(files):
#         data = np.load(data_path+"%s.npz"%(file[:-4]))
#         info_list.append(data['info'])
#     info_array = np.array(info_list)
        
    demo_mean = []
    demo_std = []
    demo_min = []
    demo_max = []
    for fold_id, sub_fold in enumerate(folds):
        print("fold%2d"%fold_id)
        sub_infos = []
        for index in sub_fold[0]:
            file_index = files.index(index)
            sub_infos.append(info_list[file_index])
        
        info_dict = {}
        for i in range(9,12):
            info_dict[i] = []
        for sub_info in sub_infos:
            for i in range(9,12):
                if sub_info[i] != -1:
                    info_dict[i].append(sub_info[i])
        # Age, height, Weight,
        ia,ih,iw=9,10,11
        demo_mean.append([np.average(info_dict[ia]),np.average(info_dict[ih]),np.average(info_dict[iw])])
        demo_std.append([np.std(info_dict[ia]),np.std(info_dict[ih]),np.std(info_dict[iw])])
        demo_min.append([np.min(info_dict[ia]),np.min(info_dict[ih]),np.min(info_dict[iw])])
        demo_max.append([np.max(info_dict[ia]),np.max(info_dict[ih]),np.max(info_dict[iw])])
    return {
        "max": demo_max,
        "min": demo_min,
        "avg": demo_mean,
        "std": demo_std,
    } 

In [141]:
def get_norm_result(samples_data,samples_mask,folds):
    print("============== norm of time-series features =================")
    samples_data = np.array(samples_data)
    samples_mask = np.array(samples_mask)

    mean_mortality = []
    std_mortality = []
    min_mortality = []
    max_mortality = []

    for fold_id, sub_fold in enumerate(folds):
        print("fold%2d"%fold_id)

        sub_data = []
        sub_mask = []
        for index in sub_fold[0]:
            file_index = files.index(index)
            sub_data.append(samples_data[file_index])
            sub_mask.append(samples_mask[file_index])

        recoders = {}
        for key in range(len(time_features)):
            recoders[key] = []

        for i,sub in enumerate(sub_data):
            for t,t_data in enumerate(sub):
                for f,f_data in enumerate(t_data):
                    if sub_mask[i][t][f] == 1:
                        recoders[f].append(f_data)

        mor_mean = []
        mor_std = []
        mor_min = []
        mor_max = []
        for key in range(len(time_features)):
            mor_mean.append(np.mean(recoders[key]))
            mor_std.append(np.std(recoders[key]))
            mor_min.append(np.min(recoders[key]))
            mor_max.append(np.max(recoders[key]))

        mean_mortality.append(mor_mean)
        std_mortality.append(mor_std)
        min_mortality.append(mor_min)
        max_mortality.append(mor_max)
    return {
        "max": max_mortality,
        "min": min_mortality,
        "avg": mean_mortality,
        "std": std_mortality,
    }

In [142]:
demo_norm1 = get_norm_demo(info_list,folds1)
regular_norm1 = get_norm_result(re_samples_data,re_samples_mask,folds1)
irregular_norm1 = get_norm_result(ir_samples_data,ir_samples_mask,folds1)

============== norm of demographic features =================
fold 0
fold 1
fold 2
fold 3
fold 4
============== norm of time-series features =================
fold 0
fold 1
fold 2
fold 3
fold 4
============== norm of time-series features =================
fold 0
fold 1
fold 2
fold 3
fold 4


In [143]:
np.savez('inhos_mortality_folds',
         fold_tvt = np.array(folds1),
         input_dim = 37,
         info_dim = 12,
         demo_norm = demo_norm1,
         regular_norm = regular_norm1,
         irregular_norm = irregular_norm1,
         time_features = time_features,
         demo_features = demo_features,
         diag_features = diag_features,
         labels = np.array(["MORTALITY_INUNIT","MORTALITY_INHOSPITAL","MORTALITY","LOS_UNIT_UNIT","LOS_UNIT_DISCHARGE"]),
        )